In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from matplotlib import image
%matplotlib qt


## Using backround extraction on averaged images to reduce temporal effects 

In [5]:
import numpy as np
import cv2 as cv


kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3))
fgbg   = cv.createBackgroundSubtractorMOG2();

frames = []
for i in range(1,4,1):
    frames.append(cv.imread(f"C:\\Users\\user1\\Documents\\Arma 3\\Screenshots\\Malden-Leopard2A6HEL_2-null%d.png"%(i),0))
frames = np.array(frames)
avgFrame = np.mean(frames,axis=0)

fgmask = fgbg.apply(avgFrame)
#fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)
cv.imshow('frame',fgmask)
k = cv.waitKey(100) & 0xff
if k == 27:
    pass


frames = []
for i in range(1,4,1):
    frames.append(cv.imread(f"C:\\Users\\user1\\Documents\\Arma 3\\Screenshots\\Malden-Leopard2A6HEL_2-%d.png"%(i),0))
frames = np.array(frames)
avgFrame = np.mean(frames,axis=0)


fgmask = fgbg.apply(avgFrame)
fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)

# Find contours
disp = cv.imread(f"C:\\Users\\user1\\Documents\\Arma 3\\Screenshots\\Malden-Leopard2A6HEL_2-3.png",1)
contours, _ = cv.findContours(fgmask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
newC = sorted(contours, key=cv.contourArea,reverse = True) # Get the bounding box of the largest contour
for c in newC:
    x, y, w, h = cv.boundingRect(c)
    print(x,y,w,h)
    cv.rectangle(disp, (x, y), (x+w, y+h), (255, 0, 0), 2)
    break

cv.imshow('rect',disp)
cv.imshow('frame',fgmask)
cv.waitKey(0)
cv.destroyAllWindows()

1610 1005 51 25


# Simple image differencing. good in case i could freeze time in the game

In [ ]:
ye = cv.imread(r"C:\Users\user1\Documents\Arma 3\Screenshots\New folder\1_with.png",1)
no = cv.imread(r"C:\Users\user1\Documents\Arma 3\Screenshots\New folder\1_blank.png",1)
# ye_g =  cv.blur(cv.cvtColor(ye, cv.COLOR_BGR2GRAY ),(3,3))
# no_g =  cv.blur(cv.cvtColor(no, cv.COLOR_BGR2GRAY ),(3,3))
ye_g =  cv.cvtColor(ye, cv.COLOR_BGR2GRAY )
no_g =  cv.cvtColor(no, cv.COLOR_BGR2GRAY )


diff = (no_g-ye_g).astype('f')
diff += diff.min(); m = diff.max()
diff *= 255
#cv.imshow('diff1',diff)

diff = np.clip(diff,0,255)
diff = diff.astype('u1')
kernel = np.ones((3, 3), np.uint8)
diff = cv.erode(diff, kernel, iterations=1)
diff = cv.dilate(diff, kernel, iterations=2)
diff = cv.blur(diff,(5,5))
cv.imshow('diff2',diff)

params = cv.SimpleBlobDetector_Params()
params.filterByArea = True
params.minArea = 100
params.filterByCircularity = False
params.filterByConvexity = False
params.filterByInertia = False

detector = cv.SimpleBlobDetector_create(params)
# Detect blobs.
keypoints = detector.detect(diff)
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv.drawKeypoints(ye, keypoints, np.array([]), (0,0,255), cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# Show keypoints
cv.imshow("Keypoints", im_with_keypoints)

# thres,th = cv.threshold(diff,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
# cv.imshow('thress',th)


contours, _ = cv.findContours(diff, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
output = ye.astype('u1')
# Get the bounding box of the largest contour
for c in contours:
    x, y, w, h = cv.boundingRect(c)
    print(x,y,w,h)
    cv.rectangle(output, (x, y), (x+w, y+h), (255, 0, 0), 2)
cv.imshow('output',output)

## Using Structural similarity

In [74]:
#  https://stackoverflow.com/a/58125368
from skimage.metrics import structural_similarity
import cv2

# Load images as grayscale

image1 = cv.imread(r"C:\Users\user1\Documents\Arma 3\Screenshots\New folder\1_with.png",0)
image2 = cv.imread(r"C:\Users\user1\Documents\Arma 3\Screenshots\New folder\1_blank.png",0)
# Compute SSIM between the two images
(score, diff) = structural_similarity(image1, image2, full=True)

# The diff image contains the actual image differences between the two images
# and is represented as a floating point data type in the range [0,1] 
# so we must convert the array to 8-bit unsigned integers in the range
# [0,255] image1 we can use it with OpenCV
diff = (diff * 255).astype("uint8")
print("Image Similarity: {:.4f}%".format(score * 100))

cv2.imshow('diff', diff)
cv2.waitKey()

Image Similarity: 98.9023%


-1